# CH5 HIGHER-ORDER FUNCTIONS

Consider two different codes for the same program.

In [1]:
let total = 0, count = 1;
while (count <= 10) {
    total += count;
    count += 1;
}
console.log(total);

55


undefined

In [2]:
//console.log(sum(range(1, 10)));

undefined

Which one is more likely to contain a bug? The 2nd one because they are expressing simpler concepts than the program as a whole, they are easier to get right.

## Abstraction
In the context of programming, these kinds of vocabularies are usually called **abstractions**. Abstractions hide details and give us the ability to talk about problems at a higher (or more abstract) level.<br>Plain functions, as we’ve seen them so far, are a good way to build abstractions. But sometimes they fall short. It is common for a program to do something a given number of times.

In [2]:
for (let i = 0; i < 10; i++) {
    console.log(i);
}

0
1
2
3
4
5
6
7
8
9


undefined

Can we abstract “doing something N times” as a function? Well, it’s easy to write a function that calls `console.log` N times.

In [4]:
function repeatLog(n) {
    for (let i = 0; i < n; i++) {
        console.log(i);
    }
}

undefined

But what if we want to do something other than logging the numbers?

In [5]:
function repeat(n, action) {
for (let i = 0; i < n; i++) {
action(i);
}}
repeat(3, console.log);

0
1
2


undefined

We don’t have to pass a predefined function to repeat . Often, it is easier to create a function value on the spot instead.

In [6]:
let labels = [];
repeat(5, i => {
    labels.push(`Unit ${i + 1}`);
});
console.log(labels);

[ 'Unit 1', 'Unit 2', 'Unit 3', 'Unit 4', 'Unit 5' ]


undefined

## Higher-Order Functions

Functions that _operate on other functions_, either by taking them as arguments or by returning them, are called **higher-order functions**. Higher-order functions allow us to abstract over actions, not just values. They come in several forms. For example, we can have functions that create new functions.

In [7]:
function greaterThan(n) {
    return m => m > n; }

let greaterThan10 = greaterThan(10);

console.log(greaterThan10(11));

true


undefined

And we can have functions that change other functions.

In [8]:
function noisy(f) {
    return (...args) => {
        console.log("calling with", args);
        let result = f(...args);
        console.log("called with", args, ", returned", result);
        return result;
    };
}
noisy(Math.min)(3, 2, 1);

calling with [ 3, 2, 1 ]
called with [ 3, 2, 1 ] , returned 1


1

We can even write functions that provide new types of control flow.

In [9]:
function unless(test, then) {
    if (!test) then();
}
repeat(6, n => {
    unless(n % 2 == 1, () => {
        console.log(n, "is even");
    });
});

0 is even
2 is even
4 is even


undefined

There is a built-in array method, `forEach` , that provides something like a for / of loop as a higher-order function.

In [10]:
["A", "B"].forEach(l => console.log(l));

A
B


undefined

## Script Data Set

One area where higher-order functions shine is data processing. To process data, we’ll need some actual data. This chapter will use a data set about scripts—writing systems such as Latin, Cyrillic, or Arabic. 

In [20]:
/*The script is run and removed because of space consumption.*/

undefined

### Filtering Arrays

In [3]:
function filter(array, test) {
    let passed = [];
    for (let element of array) {
        if (test(element)) {
            passed.push(element);
        }
    }
    return passed;
}

undefined

In [21]:
//console.log(filter(SCRIPTS, script => script.living));
// → [{name: "Adlam", ...}, ...]

undefined

The function uses the argument named test , a function value, to fill a “gap” in the computation—the process of deciding which elements to collect. Note how the filter function, rather than deleting elements from the existing array, builds up a new array with only the elements that pass the test. *This function is **pure***. It does not modify the array it is given. Like `forEach`, filter is a standard array method and the standard way to use it is:

In [19]:
console.log(SCRIPTS.filter(s => s.direction == "ttb"));

[
  {
    name: 'Mongolian',
    ranges: [
      [Array], [Array],
      [Array], [Array],
      [Array], [Array],
      [Array]
    ],
    direction: 'ttb',
    year: 1204,
    living: false,
    link: 'https://en.wikipedia.org/wiki/Mongolian_script'
  },
  {
    name: 'Phags-pa',
    ranges: [ [Array], [Array] ],
    direction: 'ttb',
    year: 1269,
    living: false,
    link: 'https://en.wikipedia.org/wiki/%27Phags-pa_script'
  },
  {
    name: 'SignWriting',
    ranges: [ [Array], [Array], [Array] ],
    direction: 'ttb',
    year: 1974,
    living: true,
    link: 'https://en.wikipedia.org/wiki/SignWriting'
  }
]


undefined

### Transforming with map

The `map` method transforms an array by applying a function to all of its elements and building a new array from the returned values. The new array will have the same length as the input array, but its content will have been mapped to a new form by the function.

In [23]:
function map(array, transform) {
    let mapped = [];
    for (let element of array) {
        mapped.push(transform(element));
    }
    return mapped;
}
let rtlScripts = SCRIPTS.filter(s => s.direction == "rtl");
console.log(map(rtlScripts, s => s.name));

[
  'Adlam',                 'Arabic',
  'Imperial Aramaic',      'Avestan',
  'Cypriot',               'Hatran',
  'Hebrew',                'Old Hungarian',
  'Kharoshthi',            'Lydian',
  'Mandaic',               'Manichaean',
  'Mende Kikakui',         'Meroitic Cursive',
  'Meroitic Hieroglyphs',  'Old North Arabian',
  'Nabataean',             'Nko',
  'Old Turkic',            'Palmyrene',
  'Inscriptional Pahlavi', 'Psalter Pahlavi',
  'Phoenician',            'Inscriptional Parthian',
  'Samaritan',             'Old South Arabian',
  'Syriac',                'Thaana'
]


undefined

Like `forEach` and `filter` , `map` is a standard array method.

### Summarizing with reduce

Another common thing to do with arrays is to compute a single value from them. Our recurring example, summing a collection of numbers, is an instance of this. The higher-order operation that represents this pattern is called `reduce` (sometimes also called `fold`). It builds a value by repeatedly taking a single element from the array and combining it with the current value. When summing numbers, you’d start with the number zero and, for each element, add that to the sum.

In [24]:
function reduce(array, combine, start) {
let current = start;
for (let element of array) {
current = combine(current, element);
}
return current;
}
console.log(reduce([1, 2, 3, 4], (a, b) => a + b, 0));

10


undefined

The standard array method `reduce` , which of course corresponds to this function, has an added convenience. If your array contains at least one element, you are allowed to leave off the start argument. The method will take the first element of the array as its start value and start reducing at the second element.

In [25]:
console.log([1, 2, 3, 4].reduce((a, b) => a + b));

10


undefined

To use reduce (twice) to find the script with the most characters, we can write something like this:

In [26]:
function characterCount(script) {
    return script.ranges.reduce((count, [from, to]) => {
        return count + (to - from);
    }, 0);
}
console.log(SCRIPTS.reduce((a, b) => {
    return characterCount(a) < characterCount(b) ? b : a; }));

{
  name: 'Han',
  ranges: [
    [ 11904, 11930 ],   [ 11931, 12020 ],
    [ 12032, 12246 ],   [ 12293, 12294 ],
    [ 12295, 12296 ],   [ 12321, 12330 ],
    [ 12344, 12348 ],   [ 13312, 19894 ],
    [ 19968, 40939 ],   [ 63744, 64110 ],
    [ 64112, 64218 ],   [ 131072, 173783 ],
    [ 173824, 177973 ], [ 177984, 178206 ],
    [ 178208, 183970 ], [ 183984, 191457 ],
    [ 194560, 195102 ]
  ],
  direction: 'ltr',
  year: -1100,
  living: true,
  link: 'https://en.wikipedia.org/wiki/Chinese_characters'
}


undefined

### Composability

Consider how we would have written the previous example (finding the biggest script) without higher-order functions. The code is not that much worse.

In [27]:
let biggest = null;
for (let script of SCRIPTS) {
    if (biggest == null || characterCount(biggest) < characterCount(script)) {
        biggest = script;
    }
}
console.log(biggest);

{
  name: 'Han',
  ranges: [
    [ 11904, 11930 ],   [ 11931, 12020 ],
    [ 12032, 12246 ],   [ 12293, 12294 ],
    [ 12295, 12296 ],   [ 12321, 12330 ],
    [ 12344, 12348 ],   [ 13312, 19894 ],
    [ 19968, 40939 ],   [ 63744, 64110 ],
    [ 64112, 64218 ],   [ 131072, 173783 ],
    [ 173824, 177973 ], [ 177984, 178206 ],
    [ 178208, 183970 ], [ 183984, 191457 ],
    [ 194560, 195102 ]
  ],
  direction: 'ltr',
  year: -1100,
  living: true,
  link: 'https://en.wikipedia.org/wiki/Chinese_characters'
}


undefined

There are a few more bindings, and the program is four lines longer.
But it is still very readable. Higher-order functions start to shine when you need to compose operations.

As an example, let’s write code that finds the average year of origin for living and dead scripts in the data set.

In [28]:
function average(array) {
    return array.reduce((a, b) => a + b) / array.length;
}

console.log(Math.round(average(
SCRIPTS.filter(s => s.living).map(s => s.year))));

console.log(Math.round(average(
SCRIPTS.filter(s => !s.living).map(s => s.year))));

1165
204


undefined

You could definitely also write this computation as one big loop.

In [30]:
let total1 = 0, count1 = 0;
for (let script of SCRIPTS) {
if (script.living) {
total1 += script.year;
count1 += 1;
}
}
console.log(Math.round(total1 / count1));

1165


undefined

The 2nd way is less readable than the first but can be faster. So we face a trade off. 

## Strings and Character Codes

One use of the data set would be figuring out what script a piece of text is using. Let’s go through a program that does this.
Remember that each script has an array of character code ranges associated with it. So given a character code, we could use a function like this to find the corresponding script (if any):

In [31]:
function characterScript(code) {
    for (let script of SCRIPTS) {
        if (script.ranges.some(([from, to]) => {
            return code >= from && code < to;
        })) {
            return script;
        }
    }
    return null;
}
console.log(characterScript(121));

{
  name: 'Latin',
  ranges: [
    [ 65, 91 ],       [ 97, 123 ],
    [ 170, 171 ],     [ 186, 187 ],
    [ 192, 215 ],     [ 216, 247 ],
    [ 248, 697 ],     [ 736, 741 ],
    [ 7424, 7462 ],   [ 7468, 7517 ],
    [ 7522, 7526 ],   [ 7531, 7544 ],
    [ 7545, 7615 ],   [ 7680, 7936 ],
    [ 8305, 8306 ],   [ 8319, 8320 ],
    [ 8336, 8349 ],   [ 8490, 8492 ],
    [ 8498, 8499 ],   [ 8526, 8527 ],
    [ 8544, 8585 ],   [ 11360, 11392 ],
    [ 42786, 42888 ], [ 42891, 42927 ],
    [ 42928, 42936 ], [ 42999, 43008 ],
    [ 43824, 43867 ], [ 43868, 43877 ],
    [ 64256, 64263 ], [ 65313, 65339 ],
    [ 65345, 65371 ]
  ],
  direction: 'ltr',
  year: -700,
  living: true,
  link: 'https://en.wikipedia.org/wiki/Latin_script'
}


undefined

The `some` method is another higher-order function. It takes a test function and tells you whether that function returns true for any of the elements in the array.

Unfortunately, obvious operations on JavaScript strings, such as getting their length through the length property and accessing their content using square brackets, deal only with code units.

In [32]:
// Two emoji characters, horse and shoe
let horseShoe = "🐴👟";
console.log(horseShoe.length);

console.log(horseShoe[0]);

console.log(horseShoe.charCodeAt(0));

console.log(horseShoe.codePointAt(0));

4
�
55357
128052


undefined

JavaScript’s `charCodeAt` method gives you a code unit, not a full character code. The `codePointAt` method, added later, does give a full Unicode character. So we could use that to get characters from a string. But the argument passed to `codePointAt` is still an index into the sequence of code units. So to run over all characters in a string, we’d still need to deal with the question of whether a character takes up one or two code units.

In [33]:
let roseDragon = "🌹🐉";
for (let char of roseDragon) {
console.log(char);
}

🌹
🐉


undefined

## Recognizing Text

We have a `characterScript` function and a way to correctly loop over characters. The next step is to count the characters that belong to each script. The following counting abstraction will be useful there:

In [5]:
function countBy(items, groupName) {
    let counts = [];
    for (let item of items) {
        let name = groupName(item);
        let known = counts.findIndex(c => c.name == name);
        if (known == -1) {
            counts.push({name, count: 1});
        } else {
            counts[known].count++;
        }
    }
    return counts;
}
console.log(countBy([1, 2, 3, 4, 5, 6], n => n >= 3.5));   

[ { name: false, count: 3 }, { name: true, count: 3 } ]


undefined

The `countBy` function expects a collection and a function that computes a group name for a given element. It uses another array method— `findIndex`. This method is somewhat like `indexOf`, but instead of looking for a specific value, it finds the first value for which the given function returns `true`. Like `indexOf` , it returns `−1` when no such element is found.

Using `countBy` , we can write the function that tells us which scripts are used in a piece of text.

In [35]:
function textScripts(text) {
    let scripts = countBy(text, char => {
        let script = characterScript(char.codePointAt(0));
        return script ? script.name : "none";
    }).filter(({name}) => name != "none");
    let total = scripts.reduce((n, {count}) => n + count, 0);
    if (total == 0) return "No scripts found";
    return scripts.map(({name, count}) => {
        return `${Math.round(count * 100 / total)}% ${name}`;
    }).join(", ");
}

undefined

In [36]:
console.log(textScripts('英国的狗英国的狗 "woof", 俄斯的狗" тяв"'));

63% Han, 21% Latin, 16% Cyrillic


undefined

The function first counts the characters by name, using `characterScript` to assign them a name and falling back to the string "none" for characters that aren’t part of any script. The `filter` call drops the entry for "none" from the resulting array since we aren’t interested in those characters. To be able to compute percentages, we first need the total number of characters that belong to a script, which we can compute with reduce . If no such characters are found, the function returns a specific string. Otherwise, it transforms the counting entries into readable strings with map and then combines them with `join`.